In [1]:
import pandas as pd

In [11]:
df = pd.read_csv('Repo_Messages_Huge_Set.csv')

In [12]:
print(df.shape)
print('Size of dataframe: {} Bytes'.format(int(df.memory_usage(index=True, deep=True).sum())))
print(df.head())

(4125158, 4)
Size of dataframe: 3221135736 Bytes
                                             message  \
0  Merge pull request #26 from plumgrid/fawadk-de...   
1  Merge pull request #48 from plumgrid/qasims-de...   
2  Update README.md\n\nAdding quick start instruc...   
3        Changed grid size for landscape orientation   
4                     Added Polish translation files   

                                              author  \
0  {'name': 'Fawad Khaliq', 'email': '4856592176e...   
1  {'name': 'Fawad Khaliq', 'email': '4856592176e...   
2  {'name': 'Muhammad Sagheer', 'email': '6e79461...   
3  {'name': 'Elman', 'email': '59c728d38dfde4d631...   
4  {'name': 'Elman', 'email': '59c728d38dfde4d631...   

                         repo  \
0  plumgrid/libnetwork-plugin   
1  plumgrid/libnetwork-plugin   
2  plumgrid/libnetwork-plugin   
3   elman22/pocket-amc-reader   
4   elman22/pocket-amc-reader   

                                            language  
0  [{'name': 'Go', 'byt